In [101]:
import pandas as pd
import requests
import sys
import time

noaa_key = "JOwIFUXjlWBqtstSqsSaKYDMzShAccQe"
usda_key = "8D5EDCA6-733D-3ACA-AF06-BA5BB3938B4F"

In [92]:
# API endpoint
url = "https://quickstats.nass.usda.gov/api/api_GET/"

params = {
    "key": usda_key,
    "commodity_desc": "CORN",
    "statisticcat_desc": "YIELD",
    "unit_desc": "BU / ACRE",
    "state_alpha": "MI",
    "year": "2022",
    "short_desc": "CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",
    "agg_level_desc": "COUNTY",
    "format": "JSON"
}

response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    print(data)  # or process the data as needed
else:
    print(f"Request failed with status code {response.status_code}")

{'data': [{'region_desc': '', 'class_desc': 'ALL CLASSES', 'watershed_desc': '', 'sector_desc': 'CROPS', 'Value': '89.7', 'asd_code': '10', 'commodity_desc': 'CORN', 'week_ending': '', 'agg_level_desc': 'COUNTY', 'domaincat_desc': 'NOT SPECIFIED', 'state_alpha': 'MI', 'prodn_practice_desc': 'ALL PRODUCTION PRACTICES', 'CV (%)': '12.8', 'begin_code': '00', 'end_code': '00', 'load_time': '2024-02-23 15:00:00.000', 'asd_desc': 'UPPER PENINSULA', 'county_ansi': '041', 'year': 2022, 'location_desc': 'MICHIGAN, UPPER PENINSULA, DELTA', 'freq_desc': 'ANNUAL', 'county_code': '041', 'unit_desc': 'BU / ACRE', 'watershed_code': '00000000', 'short_desc': 'CORN, GRAIN - YIELD, MEASURED IN BU / ACRE', 'util_practice_desc': 'GRAIN', 'country_code': '9000', 'group_desc': 'FIELD CROPS', 'state_name': 'MICHIGAN', 'state_ansi': '26', 'domain_desc': 'TOTAL', 'statisticcat_desc': 'YIELD', 'country_name': 'UNITED STATES', 'reference_period_desc': 'YEAR', 'state_fips_code': '26', 'county_name': 'DELTA', 'sou

In [96]:
pd.set_option('display.max_rows', 10) 
usda_df = pd.DataFrame(data['data'])
usda_df

,region_desc,class_desc,watershed_desc,sector_desc,Value,asd_code,commodity_desc,week_ending,agg_level_desc,domaincat_desc,...,state_ansi,domain_desc,statisticcat_desc,country_name,reference_period_desc,state_fips_code,county_name,source_desc,congr_district_code,zip_5
0,,ALL CLASSES,,CROPS,89.7,10,CORN,,COUNTY,NOT SPECIFIED,...,26,TOTAL,YIELD,UNITED STATES,YEAR,26,DELTA,SURVEY,,
1,,ALL CLASSES,,CROPS,108.9,10,CORN,,COUNTY,NOT SPECIFIED,...,26,TOTAL,YIELD,UNITED STATES,YEAR,26,DICKINSON,SURVEY,,
2,,ALL CLASSES,,CROPS,91.3,10,CORN,,COUNTY,NOT SPECIFIED,...,26,TOTAL,YIELD,UNITED STATES,YEAR,26,MENOMINEE,SURVEY,,
3,,ALL CLASSES,,CROPS,112.2,20,CORN,,COUNTY,NOT SPECIFIED,...,26,TOTAL,YIELD,UNITED STATES,YEAR,26,CHARLEVOIX,SURVEY,,
4,,ALL CLASSES,,CROPS,106,20,CORN,,COUNTY,NOT SPECIFIED,...,26,TOTAL,YIELD,UNITED STATES,YEAR,26,EMMET,SURVEY,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,,ALL CLASSES,,CROPS,142.6,90,CORN,,COUNTY,NOT SPECIFIED,...,26,TOTAL,YIELD,UNITED STATES,YEAR,26,OAKLAND,SURVEY,,
56,,ALL CLASSES,,CROPS,146.7,90,CORN,,COUNTY,NOT SPECIFIED,...,26,TOTAL,YIELD,UNITED STATES,YEAR,26,ST CLAIR,SURVEY,,
57,,ALL CLASSES,,CROPS,165.8,90,CORN,,COUNTY,NOT SPECIFIED,...,26,TOTAL,YIELD,UNITED STATES,YEAR,26,WASHTENAW,SURVEY,,
58,,ALL CLASSES,,CROPS,120.4,90,CORN,,COUNTY,NOT SPECIFIED,...,26,TOTAL,YIELD,UNITED STATES,YEAR,26,WAYNE,SURVEY,,


In [114]:
# Sate in USDA data is state_ansi
def get_raw_weather(state_ansi, county_ansi, startdate, enddate):
  url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"

  headers = {
  'token': noaa_key
  }

  # Convert startdate and enddate to datetime objects
  startdate = pd.to_datetime(startdate)
  enddate = pd.to_datetime(enddate)

  # Calculate the number of days between startdate and enddate
  num_days = (enddate - startdate).days

  # Check if the number of days is greater than 30
  if num_days > 30:
    # Split the date range into multiple requests
    requests_list = []
    current_date = startdate
    while current_date <= enddate:
      # Calculate the end date for the current request
      current_enddate = current_date + pd.DateOffset(days=30)
      if current_enddate > enddate:
        current_enddate = enddate

      # Create the parameters for the current request
      params = {
        "datasetid": "GHCND",
        "startdate": current_date.strftime("%Y-%m-%d"),
        "enddate": current_enddate.strftime("%Y-%m-%d"),
        "units": "standard",
        "limit": "1000", # 1000 is the max limit
        "locationid": f"FIPS:{state_ansi}{county_ansi}",
        'datatypeid': ['TMAX', 'TMIN']
      }

      # Make the request
      print("FIPS:{state_ansi}{county_ansi}")
      response = requests.get(url, headers=headers, params=params)

      if response.status_code == 200:
        data = response.json()
        requests_list.append(pd.DataFrame(data["results"]))
      else:
        print(response.text)

      # Move to the next date range
      current_date = current_enddate + pd.DateOffset(days=1)

    # Concatenate the results from all requests
    if requests_list:
      return pd.concat(requests_list)
    else:
      return None
  else:
    # Single request for the entire date range
    params = {
      "datasetid": "GHCND",
      "startdate": startdate.strftime("%Y-%m-%d"),
      "enddate": enddate.strftime("%Y-%m-%d"),
      "units": "standard",
      "limit": "1000",
      "locationid": f"FIPS:{state_ansi}{county_ansi}",
      'datatypeid': ['TMAX', 'TMIN']
    }
    print(f"FIPS:{state_ansi}{county_ansi}")
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
      data = response.json()
      # Return data as a DataFrame
      print(data)
      return pd.DataFrame(data["results"])
    else:
      print(response.text)
      return None

In [116]:
weather_raw = get_raw_weather("26", "109", "2022-01-01T00:00:00", "2022-12-31T00:00:00")

# Save the raw weather data to a CSV file
weather_raw.to_csv("weather_raw.csv", index=False)

FIPS:{state_ansi}{county_ansi}


KeyError: 'results'

In [ ]:
def clean_weather(df,state_ansi,county_ansi):
    # For each day in date column average the TMAX and TMIN into their own columns across all stations
    df["date"] = pd.to_datetime(df["date"])
    df["TMAX"] = df["value"].where(df["datatype"] == "TMAX")
    df["TMIN"] = df["value"].where(df["datatype"] == "TMIN")
    
    df = df.groupby("date").agg({"TMAX": "mean", "TMIN": "mean"}).reset_index()

    # add state and county columns
    df["state"] = state_ansi
    df["county"] = county_ansi
    
    return df

In [91]:
weather = clean_weather(weather_raw, "26", "145")

# Print the first 25 rows of the weather DataFrame
weather.head(25)
weather.to_csv("weather.csv", index=False)

In [115]:
# for each state_ansi county_ansi combo in the USDA data, get the weather data

def get_weather_data(usda_df):
    weather_data = []
    for index, row in usda_df.iterrows():
        state_ansi = row["state_ansi"]
        county_ansi = row["county_ansi"]
        weather_raw = get_raw_weather(state_ansi, county_ansi, "2022-12-20T00:00:00", "2022-12-31T00:00:00")
        if weather_raw is not None:
            weather = clean_weather(weather_raw, state_ansi, county_ansi)
            weather_data.append(weather)
    return pd.concat(weather_data)

weather_data = get_weather_data(usda_df)
weather_data.to_csv("weather_data.csv", index=False)

FIPS:26041
{'metadata': {'resultset': {'offset': 1, 'count': 68, 'limit': 1000}}, 'results': [{'date': '2022-12-20T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:USC00203078', 'attributes': ',,7,', 'value': 30.0}, {'date': '2022-12-20T00:00:00', 'datatype': 'TMIN', 'station': 'GHCND:USC00203078', 'attributes': ',,7,', 'value': 20.0}, {'date': '2022-12-20T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:USC00206840', 'attributes': ',,7,', 'value': 25.0}, {'date': '2022-12-20T00:00:00', 'datatype': 'TMIN', 'station': 'GHCND:USC00206840', 'attributes': ',,7,', 'value': 20.0}, {'date': '2022-12-20T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:USW00094853', 'attributes': ',,W,', 'value': 24.0}, {'date': '2022-12-20T00:00:00', 'datatype': 'TMIN', 'station': 'GHCND:USW00094853', 'attributes': ',,W,', 'value': 4.0}, {'date': '2022-12-21T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:USC00203078', 'attributes': ',,7,', 'value': 24.0}, {'date': '2022-12-21T00:00:00', 'datatype': 'TMIN', '

KeyError: 'results'